# Dataset

In [6]:
import os
import re
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

def analyze_dataset_quality(dataset_dir, dataset_name="Dataset"):
    """Comprehensive analysis of Market1501 format dataset"""
    
    # Market1501 naming pattern: PPPP_CC_SSSSSS.jpg
    pattern = re.compile(r'(\d+)_c(\d+)_')  # Adjusted for your pattern
    market_pattern = re.compile(r'(\d{4})_(\d{2})_(\d{6})\.jpg')  # Standard Market1501
    
    id_counts = defaultdict(int)
    cam_counts = defaultdict(int)
    id_cam_pairs = defaultdict(set)
    cam_id_pairs = defaultdict(set)
    
    images = [f for f in os.listdir(dataset_dir) if f.endswith(('.jpg', '.png'))]
    
    for img in images:
        # Try Market1501 pattern first
        m = market_pattern.search(img)
        if m:
            person_id = int(m.group(1))
            camera_id = int(m.group(2))
        else:
            # Try your custom pattern
            m = pattern.search(img)
            if m:
                person_id = int(m.group(1))
                camera_id = int(m.group(2))
            else:
                continue
        
        id_counts[person_id] += 1
        cam_counts[camera_id] += 1
        id_cam_pairs[person_id].add(camera_id)
        cam_id_pairs[camera_id].add(person_id)
    
    # Calculate statistics
    id_counts_list = list(id_counts.values())
    
    print(f"\n{'='*50}")
    print(f"{dataset_name} Analysis")
    print(f"{'='*50}")
    print(f"Total images: {len(images)}")
    print(f"Unique person IDs: {len(id_counts)}")
    print(f"Unique cameras: {len(cam_counts)}")
    
    if id_counts:
        print(f"\nImages per person ID:")
        print(f"  Average: {np.mean(id_counts_list):.1f}")
        print(f"  Median: {np.median(id_counts_list):.1f}")
        print(f"  Min: {min(id_counts_list)}")
        print(f"  Max: {max(id_counts_list)}")
        print(f"  Std: {np.std(id_counts_list):.1f}")
        
        # Distribution analysis
        low_count_ids = [id for id, count in id_counts.items() if count < 5]
        medium_count_ids = [id for id, count in id_counts.items() if 5 <= count < 20]
        high_count_ids = [id for id, count in id_counts.items() if count >= 20]
        
        print(f"\nDistribution:")
        print(f"  IDs with < 5 images: {len(low_count_ids)} ({len(low_count_ids)/len(id_counts)*100:.1f}%)")
        print(f"  IDs with 5-19 images: {len(medium_count_ids)} ({len(medium_count_ids)/len(id_counts)*100:.1f}%)")
        print(f"  IDs with >= 20 images: {len(high_count_ids)} ({len(high_count_ids)/len(id_counts)*100:.1f}%)")
        
        # Camera distribution
        print(f"\nCamera distribution:")
        for cam_id in sorted(cam_counts.keys()):
            print(f"  Camera {cam_id}: {cam_counts[cam_id]} images, {len(cam_id_pairs[cam_id])} unique persons")
        
        # Cross-camera analysis
        multi_cam_ids = [id for id, cams in id_cam_pairs.items() if len(cams) > 1]
        print(f"\nCross-camera statistics:")
        print(f"  IDs appearing in multiple cameras: {len(multi_cam_ids)} ({len(multi_cam_ids)/len(id_counts)*100:.1f}%)")
        
        if multi_cam_ids:
            cam_counts_per_id = [len(id_cam_pairs[id]) for id in multi_cam_ids]
            print(f"  Average cameras per multi-cam ID: {np.mean(cam_counts_per_id):.1f}")
    
    return id_counts, cam_counts, id_cam_pairs

def plot_distribution(id_counts, title="Images per Person Distribution"):
    """Plot histogram of images per person"""
    plt.figure(figsize=(10, 6))
    counts = list(id_counts.values())
    plt.hist(counts, bins=50, edgecolor='black')
    plt.xlabel('Number of Images')
    plt.ylabel('Number of Persons')
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.show()

def check_data_consistency(train_dir, query_dir, gallery_dir):
    """Check for data consistency across splits"""
    print(f"\n{'='*50}")
    print("Data Consistency Check")
    print(f"{'='*50}")
    
    # Get person IDs from each split
    train_ids = set()
    query_ids = set()
    gallery_ids = set()
    
    pattern = re.compile(r'(\d+)_')
    
    for img in os.listdir(train_dir):
        m = pattern.search(img)
        if m:
            train_ids.add(int(m.group(1)))
    
    for img in os.listdir(query_dir):
        m = pattern.search(img)
        if m:
            query_ids.add(int(m.group(1)))
    
    for img in os.listdir(gallery_dir):
        m = pattern.search(img)
        if m:
            gallery_ids.add(int(m.group(1)))
    
    # Check overlaps
    train_query_overlap = train_ids & query_ids
    train_gallery_overlap = train_ids & gallery_ids
    query_gallery_overlap = query_ids & gallery_ids
    
    print(f"Train IDs: {len(train_ids)}")
    print(f"Query IDs: {len(query_ids)}")
    print(f"Gallery IDs: {len(gallery_ids)}")
    
    print(f"\nOverlaps (should be non-zero for proper evaluation):")
    print(f"  Train ∩ Query: {len(train_query_overlap)}")
    print(f"  Train ∩ Gallery: {len(train_gallery_overlap)}")
    print(f"  Query ∩ Gallery: {len(query_gallery_overlap)} (should equal Query IDs)")
    
    # Check if query IDs are subset of gallery IDs
    if query_ids.issubset(gallery_ids):
        print("✓ All query IDs exist in gallery (correct)")
    else:
        missing = query_ids - gallery_ids
        print(f"✗ {len(missing)} query IDs missing from gallery: {list(missing)[:5]}...")
    
    return train_ids, query_ids, gallery_ids

def generate_reid_statistics(base_dir):
    """Generate comprehensive statistics for the converted dataset"""
    train_dir = os.path.join(base_dir, "bounding_box_train")
    query_dir = os.path.join(base_dir, "query")
    gallery_dir = os.path.join(base_dir, "bounding_box_test")
    
    # Analyze each split
    train_stats = analyze_dataset_quality(train_dir, "TRAIN SET")
    query_stats = analyze_dataset_quality(query_dir, "QUERY SET")
    gallery_stats = analyze_dataset_quality(gallery_dir, "GALLERY SET")
    
    # Check consistency
    check_data_consistency(train_dir, query_dir, gallery_dir)
    
    # Generate summary report
    print(f"\n{'='*50}")
    print("SUMMARY REPORT")
    print(f"{'='*50}")
    
    total_images = len(os.listdir(train_dir)) + len(os.listdir(query_dir)) + len(os.listdir(gallery_dir))
    print(f"Total images in dataset: {total_images}")
    
    # Save statistics to file
    stats_file = os.path.join(base_dir, "dataset_statistics.txt")
    with open(stats_file, 'w') as f:
        f.write("CCVID to Market1501 Conversion Statistics\n")
        f.write("="*50 + "\n")
        f.write(f"Total images: {total_images}\n")
        f.write(f"Train images: {len(os.listdir(train_dir))}\n")
        f.write(f"Query images: {len(os.listdir(query_dir))}\n")
        f.write(f"Gallery images: {len(os.listdir(gallery_dir))}\n")
    
    print(f"\nStatistics saved to: {stats_file}")

# Main execution
if __name__ == "__main__":
    base_dir = "/home/ika/yzlm/TwinProject/CCVID"
    
    # Run comprehensive analysis
    generate_reid_statistics(base_dir)
    
    # Optional: Plot distributions
    # train_dir = os.path.join(base_dir, "bounding_box_train")
    # train_stats, _, _ = analyze_dataset_quality(train_dir, "TRAIN SET")
    # plot_distribution(train_stats, "Train Set: Images per Person")


TRAIN SET Analysis
Total images: 4984
Unique person IDs: 75
Unique cameras: 6

Images per person ID:
  Average: 66.5
  Median: 66.0
  Min: 65
  Max: 70
  Std: 1.1

Distribution:
  IDs with < 5 images: 0 (0.0%)
  IDs with 5-19 images: 0 (0.0%)
  IDs with >= 20 images: 75 (100.0%)

Camera distribution:
  Camera 1: 836 images, 75 unique persons
  Camera 2: 831 images, 75 unique persons
  Camera 3: 832 images, 75 unique persons
  Camera 4: 827 images, 75 unique persons
  Camera 5: 826 images, 75 unique persons
  Camera 6: 832 images, 75 unique persons

Cross-camera statistics:
  IDs appearing in multiple cameras: 75 (100.0%)
  Average cameras per multi-cam ID: 6.0

QUERY SET Analysis
Total images: 1899
Unique person IDs: 151
Unique cameras: 6

Images per person ID:
  Average: 12.6
  Median: 9.0
  Min: 9
  Max: 18
  Std: 4.4

Distribution:
  IDs with < 5 images: 0 (0.0%)
  IDs with 5-19 images: 151 (100.0%)
  IDs with >= 20 images: 0 (0.0%)

Camera distribution:
  Camera 1: 453 images, 151

# Train 

In [6]:
!tao model re_identification train -e /home/ika/yzlm/TwinProject/CCVID/train_with_val.yaml

2025-07-27 19:53:25,843 [TAO Toolkit] [INFO] root 160: Registry: ['nvcr.io']
2025-07-27 19:53:25,888 [TAO Toolkit] [INFO] nvidia_tao_cli.components.instance_handler.local_instance 360: Running command in container: nvcr.io/nvidia/tao/tao-toolkit:6.0.0-pyt
2025-07-27 19:53:25,896 [TAO Toolkit] [WARNING] nvidia_tao_cli.components.docker_handler.docker_handler 295: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/ika/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
2025-07-27 19:53:25,897 [TAO Toolkit] [INFO] nvidia_tao_cli.components.docker_handler.docker_handler 308: Printing tty value True
2025-07-27 16:53:28,445 [TAO Toolkit] [INFO] matplotlib.font_manager 1639: generated new fontManager
sys:1: UserWarning: 
'train_with_val.yaml' is validated against ConfigStore schema with the same name.
This 

# Evaluate


In [2]:
!tao model re_identification export \
    -e /home/ika/yzlm/TwinProject/CCVID/export.yaml \
    dataset.num_classes=75

2025-07-28 17:57:28,638 [TAO Toolkit] [INFO] root 160: Registry: ['nvcr.io']
2025-07-28 17:57:28,682 [TAO Toolkit] [INFO] nvidia_tao_cli.components.instance_handler.local_instance 360: Running command in container: nvcr.io/nvidia/tao/tao-toolkit:6.0.0-pyt
2025-07-28 17:57:28,691 [TAO Toolkit] [WARNING] nvidia_tao_cli.components.docker_handler.docker_handler 295: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/ika/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
2025-07-28 17:57:28,691 [TAO Toolkit] [INFO] nvidia_tao_cli.components.docker_handler.docker_handler 308: Printing tty value True
2025-07-28 14:57:31,201 [TAO Toolkit] [INFO] matplotlib.font_manager 1639: generated new fontManager
sys:1: UserWarning: 
'export.yaml' is validated against ConfigStore schema with the same name.
This behavior

In [6]:
import os
import random

def split_gallery_symlink(src_folder, dst_root, n_parts=2):
    files = [f for f in os.listdir(src_folder) if f.endswith('.jpg') or f.endswith('.png')]
    random.shuffle(files)  # Rastgele karıştır
    total = len(files)
    part_size = (total + n_parts - 1) // n_parts

    for i in range(n_parts):
        part_dir = os.path.join(dst_root, f"part_{i+1}")
        os.makedirs(part_dir, exist_ok=True)
        part_files = files[i*part_size : (i+1)*part_size]
        for f in part_files:
            src = os.path.join(src_folder, f)
            dst = os.path.join(part_dir, f)
            if not os.path.exists(dst):
                os.symlink(src, dst)
        print(f"Part {i+1}: {len(part_files)} symlinks created.")

# Kullanım:
split_gallery_symlink(
    src_folder="/home/ika/yzlm/TwinProject/CCVID/data/bounding_box_test",
    dst_root="/home/ika/yzlm/TwinProject/CCVID/data/bounding_box_test_split",
    n_parts=15
)

Part 1: 7495 symlinks created.
Part 2: 7495 symlinks created.
Part 3: 7495 symlinks created.
Part 4: 7495 symlinks created.
Part 5: 7495 symlinks created.
Part 6: 7495 symlinks created.
Part 7: 7495 symlinks created.
Part 8: 7495 symlinks created.
Part 9: 7495 symlinks created.
Part 10: 7495 symlinks created.
Part 11: 7495 symlinks created.
Part 12: 7495 symlinks created.
Part 13: 7495 symlinks created.
Part 14: 7495 symlinks created.
Part 15: 7491 symlinks created.


In [7]:
import os

def split_folder_symlink(src_folder, dst_root, n_parts=5):
    files = [f for f in os.listdir(src_folder) if f.endswith('.jpg') or f.endswith('.png')]
    files.sort()
    total = len(files)
    part_size = (total + n_parts - 1) // n_parts

    for i in range(n_parts):
        part_dir = os.path.join(dst_root, f"part_{i+1}")
        os.makedirs(part_dir, exist_ok=True)
        part_files = files[i*part_size : (i+1)*part_size]
        for f in part_files:
            os.symlink(os.path.join(src_folder, f), os.path.join(part_dir, f))
        print(f"Part {i+1}: {len(part_files)} symlinks created.")

# Kullanım:
src_folder = "/home/ika/yzlm/TwinProject/CCVID/data/query"
dst_root = "/home/ika/yzlm/TwinProject/CCVID/data/query_splits/"
split_folder_symlink(src_folder, dst_root, n_parts=30)

FileExistsError: [Errno 17] File exists: '/home/ika/yzlm/TwinProject/CCVID/data/query/0001_c10s1_01_00.jpg' -> '/home/ika/yzlm/TwinProject/CCVID/data/query_splits/part_1/0001_c10s1_01_00.jpg'

In [8]:
!tao model re_identification evaluate \
    -e /home/ika/yzlm/TwinProject/CCVID/train_with_val.yaml \
    evaluate.checkpoint=/home/ika/yzlm/TwinProject/CCVID/results/train/model_epoch_029_step_50729.pth \
    evaluate.query_dataset=/home/ika/yzlm/TwinProject/CCVID/data/query_splits/part_1 \
    evaluate.test_dataset=/home/ika/yzlm/TwinProject/CCVID/data/bounding_box_test_split/part_1   \
    re_ranking.re_ranking=True


2025-07-27 20:51:46,716 [TAO Toolkit] [INFO] root 160: Registry: ['nvcr.io']
2025-07-27 20:51:46,762 [TAO Toolkit] [INFO] nvidia_tao_cli.components.instance_handler.local_instance 360: Running command in container: nvcr.io/nvidia/tao/tao-toolkit:6.0.0-pyt
2025-07-27 20:51:46,772 [TAO Toolkit] [WARNING] nvidia_tao_cli.components.docker_handler.docker_handler 295: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/ika/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
2025-07-27 20:51:46,772 [TAO Toolkit] [INFO] nvidia_tao_cli.components.docker_handler.docker_handler 308: Printing tty value True
2025-07-27 17:51:49,486 [TAO Toolkit] [INFO] matplotlib.font_manager 1639: generated new fontManager
sys:1: UserWarning: 
'train_with_val.yaml' is validated against ConfigStore schema with the same name.
This 

In [4]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import onnxruntime as ort
from torchvision import transforms
import torch
import pickle

class ReIDImageDataset:
    def __init__(self, img_dir):
        self.img_dir = img_dir
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith('.jpg') or f.endswith('.png')]
        self.img_files.sort()
        self.transform = transforms.Compose([
            transforms.Resize((256, 128)),
            transforms.ToTensor(),
            transforms.Normalize([0.444, 0.438, 0.457], [0.288, 0.280, 0.275])
        ])
    
    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        pid = int(self.img_files[idx].split('_')[0])
        return img, pid, self.img_files[idx]

def save_features(features, pids, fnames, save_path):
    """Save extracted features to disk"""
    data = {
        'features': features,
        'pids': pids,
        'fnames': fnames
    }
    # Use numpy's compressed format for efficiency
    np.savez_compressed(save_path, **data)
    print(f"Features saved to: {save_path}")

def load_features(load_path):
    """Load features from disk"""
    data = np.load(load_path)
    features = data['features']
    pids = data['pids']
    fnames = data['fnames']
    print(f"Features loaded from: {load_path}")
    return features, pids, fnames

def extract_features_onnx(onnx_path, dataset, batch_size=32, save_path=None):
    """Extract features with option to save"""
    
    # Check if features already exist
    if save_path and os.path.exists(save_path):
        print(f"Loading existing features from: {save_path}")
        return load_features(save_path)
    
    # Extract features
    session = ort.InferenceSession(
        onnx_path,
        providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
    )   
    input_name = session.get_inputs()[0].name
    features, pids, fnames = [], [], []
    
    for i in tqdm(range(0, len(dataset), batch_size)):
        batch_imgs = []
        batch_pids = []
        batch_fnames = []
        for j in range(i, min(i+batch_size, len(dataset))):
            img, pid, fname = dataset[j]
            batch_imgs.append(img.numpy())
            batch_pids.append(pid)
            batch_fnames.append(fname)
        batch_imgs = np.stack(batch_imgs)
        
        # ONNX inference
        feats = session.run(None, {input_name: batch_imgs.astype(np.float32)})[0]
        
        # L2 normalize
        feats = feats / np.linalg.norm(feats, axis=1, keepdims=True)
        
        features.append(feats)
        pids.extend(batch_pids)
        fnames.extend(batch_fnames)
    
    features = np.vstack(features)
    pids = np.array(pids)
    
    # Save if path provided
    if save_path:
        save_features(features, pids, fnames, save_path)
    
    return features, pids, fnames

def evaluate_gpu_memory_efficient(query_features, gallery_features, query_pids, gallery_pids,
                                  batch_size=512, topk=(1, 5, 10)):
    """Memory-efficient and GPU-accelerated ReID evaluation."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Evaluation device: {device}")

    # Tensörleri GPU'ya taşı
    qf = torch.from_numpy(query_features).to(device)
    gf = torch.from_numpy(gallery_features).to(device)
    q_pids = torch.from_numpy(query_pids).to(device)
    g_pids = torch.from_numpy(gallery_pids).to(device)

    num_query = qf.shape[0]
    max_rank = max(topk)

    # Toplam metrikleri tutmak için
    total_cmc = torch.zeros(max_rank, device=device)
    total_ap = 0.0
    valid_queries = 0

    for i in tqdm(range(0, num_query, batch_size), desc="Evaluating Batches"):
        batch_qf = qf[i : i + batch_size]
        batch_q_pids = q_pids[i : i + batch_size]

        # GPU üzerinde mesafeleri hesapla
        distmat = torch.cdist(batch_qf, gf, p=2)

        # Sıralanmış indisleri al
        indices = torch.argsort(distmat, dim=1)

        # GPU üzerinde eşleşmeleri bul
        matches = (g_pids[indices] == batch_q_pids.view(-1, 1)).int()

        # Geçerli sorguları bul (galeride en az bir eşleşmesi olanlar)
        num_rel = matches.sum(dim=1)
        has_match = num_rel > 0
        
        if not has_match.any():
            continue
            
        valid_queries += has_match.sum().item()

        # AP (Average Precision) hesapla
        positions = torch.arange(1, matches.shape[1] + 1, device=device).expand_as(matches)
        precision = matches.cumsum(dim=1) / positions
        ap = (precision * matches).sum(dim=1) / num_rel.clamp(min=1)
        total_ap += ap[has_match].sum().item()

        # CMC (Cumulative Matching Characteristics) hesapla
        cmc = matches.cumsum(dim=1)
        cmc[cmc > 1] = 1
        total_cmc += cmc[has_match, :max_rank].sum(dim=0)

    # Final metrikleri hesapla
    mAP = total_ap / valid_queries if valid_queries > 0 else 0.0
    cmc_scores = (total_cmc / valid_queries).cpu().numpy()

    final_scores = [cmc_scores[k-1] for k in topk]
    return mAP, final_scores


if __name__ == "__main__":
    print(f"PyTorch CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
    # Paths
    onnx_path = "/home/ika/yzlm/TwinProject/CCVID/results/exported/model24.onnx"
    query_dir = "/home/ika/yzlm/TwinProject/CCVID/data/query"
    gallery_dir = "/home/ika/yzlm/TwinProject/CCVID/data/bounding_box_test"
    
    # Feature save paths
    query_features_path = "/home/ika/yzlm/TwinProject/CCVID/features/query_features.npz"
    gallery_features_path = "/home/ika/yzlm/TwinProject/CCVID/features/gallery_features.npz"
    
    # Create features directory if not exists
    os.makedirs(os.path.dirname(query_features_path), exist_ok=True)
    
    # Parameters
    feature_batch_size = 256  # For ONNX inference
    eval_batch_size = 1024    # For GPU distance computation
    
    # Load datasets
    print("\nLoading datasets...")
    query_dataset = ReIDImageDataset(query_dir)
    gallery_dataset = ReIDImageDataset(gallery_dir)
    print(f"Query: {len(query_dataset)} images")
    print(f"Gallery: {len(gallery_dataset)} images")
    
    # Extract or load features
    print("\nExtracting/Loading query features...")
    query_features, query_pids, query_fnames = extract_features_onnx(
        onnx_path, query_dataset, 
        batch_size=feature_batch_size,
        save_path=query_features_path
    )
    
    print("\nExtracting/Loading gallery features...")
    gallery_features, gallery_pids, gallery_fnames = extract_features_onnx(
        onnx_path, gallery_dataset, 
        batch_size=feature_batch_size,
        save_path=gallery_features_path
    )
    
    print(f"\nFeature shapes:")
    print(f"Query: {query_features.shape}")
    print(f"Gallery: {gallery_features.shape}")
    


PyTorch CUDA available: True
GPU: NVIDIA GeForce RTX 5070
VRAM: 11.5 GB

Loading datasets...
Query: 116799 images
Gallery: 112421 images

Extracting/Loading query features...
Loading existing features from: /home/ika/yzlm/TwinProject/CCVID/features/query_features.npz
Features loaded from: /home/ika/yzlm/TwinProject/CCVID/features/query_features.npz

Extracting/Loading gallery features...
Loading existing features from: /home/ika/yzlm/TwinProject/CCVID/features/gallery_features.npz
Features loaded from: /home/ika/yzlm/TwinProject/CCVID/features/gallery_features.npz

Feature shapes:
Query: (116799, 256)
Gallery: (112421, 256)


In [3]:
# Bellek verimli GPU ile değerlendirme
mAP, cmc_scores = evaluate_gpu_memory_efficient(
    query_features, gallery_features,
    query_pids, gallery_pids,
    batch_size=128,
    topk=(1, 5, 10)
)

print(f"mAP: {mAP:.4f}")
print(f"Rank-1: {cmc_scores[0]:.4f}")
print(f"Rank-5: {cmc_scores[1]:.4f}")
print(f"Rank-10: {cmc_scores[2]:.4f}")

# Optional: Save results
results = {
    'mAP': mAP,
    'rank1': cmc_scores[0],
    'rank5': cmc_scores[1],
    'rank10': cmc_scores[2]
}

"""results_path = "/home/ika/yzlm/TwinProject/CCVID/features/evaluation_results.pkl"
with open(results_path, 'wb') as f:
    pickle.dump(results, f)
print(f"\nResults saved to: {results_path}")"""

Evaluation device: cuda


Evaluating Batches: 100%|██████████| 913/913 [00:10<00:00, 84.91it/s]

mAP: 0.3319
Rank-1: 0.6199
Rank-5: 0.6811
Rank-10: 0.7065


'results_path = "/home/ika/yzlm/TwinProject/CCVID/features/evaluation_results.pkl"\nwith open(results_path, \'wb\') as f:\n    pickle.dump(results, f)\nprint(f"\nResults saved to: {results_path}")'